In [3]:
!pip install split-folders

In [6]:
import splitfolders
splitfolders.ratio('/kaggle/input/eye-diseases-classification/dataset',output='outputfolders',seed=1337,ratio=(0.8,0.2))

Copying files: 4217 files [00:44, 95.76 files/s] 


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.layers import Flatten,Dense
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [10]:
image_size=[224,224]
traindata=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
testdata=ImageDataGenerator(rescale=1./255)

In [11]:
trainset=traindata.flow_from_directory('/kaggle/working/outputfolders/train',target_size=image_size,batch_size=64,class_mode='categorical')
testset=testdata.flow_from_directory('/kaggle/working/outputfolders/val',target_size=image_size,batch_size=64,class_mode='categorical')


Found 3372 images belonging to 4 classes.
Found 845 images belonging to 4 classes.


In [12]:
VGG19=VGG19(input_shape=image_size+[3],weights='imagenet',include_top=False)

80134624/80134624 [==============================] - 0s 0us/step


In [13]:
for layer in VGG19.layers:
    layer.trainable=False


In [14]:
x=Flatten()(VGG19.output)
predict=Dense(4,activation='softmax')(x)
model=Model(inputs=VGG19.input,outputs=predict)

In [15]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [16]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [17]:
history=model.fit(trainset,validation_data=testset,epochs=50,steps_per_epoch=len(trainset),validation_steps=len(testset))

Epoch 1/50
53/53 [==============================] - 91s 1s/step - loss: 1.0787 - accuracy: 0.6189 - val_loss: 0.6005 - val_accuracy: 0.7645
Epoch 2/50
53/53 [==============================] - 69s 1s/step - loss: 0.5714 - accuracy: 0.7767 - val_loss: 0.6958 - val_accuracy: 0.6994
Epoch 3/50
53/53 [==============================] - 68s 1s/step - loss: 0.5256 - accuracy: 0.7933 - val_loss: 0.5036 - val_accuracy: 0.8213
Epoch 4/50
53/53 [==============================] - 67s 1s/step - loss: 0.4917 - accuracy: 0.7951 - val_loss: 0.6498 - val_accuracy: 0.7408
Epoch 5/50
53/53 [==============================] - 67s 1s/step - loss: 0.4712 - accuracy: 0.8149 - val_loss: 0.5934 - val_accuracy: 0.7645
Epoch 6/50
53/53 [==============================] - 67s 1s/step - loss: 0.4310 - accuracy: 0.8301 - val_loss: 0.5497 - val_accuracy: 0.7751
Epoch 7/50
53/53 [==============================] - 67s 1s/step - loss: 0.4203 - accuracy: 0.8351 - val_loss: 0.4655 - val_accuracy: 0.8189
Epoch 8/50
53/53 [==

In [18]:
model.save('model.h5')

In [19]:
model=load_model('/kaggle/working/model.h5')


In [26]:
img=image.load_img(r'/kaggle/working/outputfolders/val/normal/3208_right.jpg',target_size=image_size)
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
pred=model.predict(x)
predict=np.argmax(pred,axis=1)
index=['diabetic_retinopathy','cataract','glaucoma','normal']
result=str(index[predict[0]])


1/1 [==============================] - 0s 21ms/step


In [27]:
result

'normal'